In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mitbih1/117.atr
/kaggle/input/mitbih1/103.dat
/kaggle/input/mitbih1/106.hea
/kaggle/input/mitbih1/113.dat
/kaggle/input/mitbih1/202.dat
/kaggle/input/mitbih1/217.dat
/kaggle/input/mitbih1/111.atr
/kaggle/input/mitbih1/208.hea
/kaggle/input/mitbih1/219.dat
/kaggle/input/mitbih1/104.atr
/kaggle/input/mitbih1/201.dat
/kaggle/input/mitbih1/201.atr
/kaggle/input/mitbih1/121.hea
/kaggle/input/mitbih1/219.atr
/kaggle/input/mitbih1/102.dat
/kaggle/input/mitbih1/116.hea
/kaggle/input/mitbih1/231.dat
/kaggle/input/mitbih1/111.hea
/kaggle/input/mitbih1/209.hea
/kaggle/input/mitbih1/122.atr
/kaggle/input/mitbih1/203.hea
/kaggle/input/mitbih1/212.atr
/kaggle/input/mitbih1/232.dat
/kaggle/input/mitbih1/100.hea
/kaggle/input/mitbih1/234.hea
/kaggle/input/mitbih1/214.dat
/kaggle/input/mitbih1/119.hea
/kaggle/input/mitbih1/228.dat
/kaggle/input/mitbih1/107.dat
/kaggle/input/mitbih1/108.hea
/kaggle/input/mitbih1/123.hea
/kaggle/input/mitbih1/121.dat
/kaggle/input/mitbih1/105.atr
/kaggle/in

In [2]:
import numpy as np
import wfdb
import matplotlib.pyplot as plt
from scipy.signal import resample
import glob
import pywt

#--------------------小波去噪-----------------
def WTfilt_1d(sig):
    """
    对信号进行小波变换滤波
    :param sig: 输入信号，1-d array
    :return: 小波滤波后的信号，1-d array
    """
    coeffs = pywt.wavedec(sig, 'db6', level=9)
    coeffs[-1] = np.zeros(len(coeffs[-1]))
    coeffs[-2] = np.zeros(len(coeffs[-2]))
    coeffs[0] = np.zeros(len(coeffs[0]))
    sig_filt = pywt.waverec(coeffs, 'db6')
    return sig_filt    


#-------------------------心拍截取-------------------
def heartbeat(file0):
    '''
    file0:下载的MITAB数据
    
    '''
    N_Seg=[]; SVEB_Seg=[];  VEB_Seg=[]; F_Seg=[] ; Q_Seg=[];
    #--------去掉指定的四个导联的头文件---------
    De_file=[panth[:-1]+'\\102.hea',panth[:-1]+'\\104.hea',panth[:-1]+'\\107.hea',panth[:-1]+'\\217.hea']
    file=list(set(file0).difference(set(De_file)))
    
    for f in range(len(file)) :
        annotation= wfdb.rdann(panth+file[f][-7:-4],'atr')
        record_name=annotation.record_name    #读取记录名称
        Record=wfdb.rdsamp(panth+record_name)[0][:,0] #一般只取一个导联
        record=WTfilt_1d(Record)         #小波去噪
        label=annotation.symbol  #心拍标签列表
        label_index=annotation.sample   #标签索引列表
        for j in range(len(label_index)):
            if label_index[j]>=144  and (label_index[j]+180)<=650000:
                if label[j]=='N' or label[j]=='.' or label[j]=='L' or label[j]=='R' or label[j]=='e' or label[j]=='j':
                    Seg=record[label_index[j]-144:label_index[j]+180]#R峰的前0.4s和后0.5s
                    segment=resample(Seg,251, axis=0)  #重采样到251
                    N_Seg.append(segment)
                    
                if label[j]=='A' or label[j]=='a' or label[j]=='J' or label[j]=='S':
                    
                    Seg=record[label_index[j]-144:label_index[j]+180]
                    segment=resample(Seg,251, axis=0) 
                    SVEB_Seg.append(segment)
                    
                if label[j]=='V' or label[j]=='E':
                   
                    Seg=record[label_index[j]-144:label_index[j]+180]
                    segment=resample(Seg,251, axis=0)  
                    VEB_Seg.append(segment)
                    
                if label[j]=='F':
                    
                    Seg=record[label_index[j]-144:label_index[j+1]+180]
                    segment=resample(Seg,251, axis=0)  
                    F_Seg.append(segment)
                if  label[j]=='/' or label[j]=='f' or label[j]=='Q':
                    
                    Seg=record[label_index[j]-144:label_index[j]+180]
                    segment=resample(Seg,251, axis=0)  
                    Q_Seg.append(segment)
                    
    N_segement=np.array(N_Seg)
    SVEB_segement=np.array(SVEB_Seg)
    VEB_segement=np.array(VEB_Seg)
    F_segement=np.array(F_Seg)
    Q_segement=np.array(Q_Seg)
    
    label_N=np.zeros(N_segement.shape[0])
    label_SVEB=np.ones(SVEB_segement.shape[0])
    label_VEB=np.ones(VEB_segement.shape[0])*2
    label_F=np.ones(F_segement.shape[0])*3
    label_Q=np.ones(Q_segement.shape[0])*4
                    
    Data=np.concatenate((N_segement,SVEB_segement,VEB_segement,F_segement),axis=0)
    Label=np.concatenate((label_N,label_SVEB,label_VEB,label_F,),axis=0)
    
    return  Data, Label

#-----------------------心拍截取和保存---------------------
#建议一次性截取和保存，不需要重复操作，下次训练和测试的时候，直接load
panth='/kaggle/input/mitbih1/'
file = glob.glob(panth+'*.hea')
Data, Label=heartbeat(file)

Data=np.save('/kaggle/working/'+'Data',Data)
Label=np.save('/kaggle/working/'+'Label',Label)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, optimizers, datasets, Sequential
from tensorflow.keras.layers import MaxPooling1D, Conv1D, Conv2D, AveragePooling1D, MaxPool1D, add, Flatten, Dense, Concatenate, Activation
from keras import regularizers
import os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
tf.random.set_seed(2345)

def preprocess(x, y):
    """
    x is a simple image, not a batch
    """
    x = tf.cast(x, dtype=tf.float32)
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=4)
    return x, y

#导入数据
data = np.load('/kaggle/working/Data.npy')
label = np.load('/kaggle/working/Label.npy')

data = tf.expand_dims(data, axis=2)
db_num = int(data.shape[0])
train_num = int(data.shape[0]/5*4)
idx = tf.range(db_num)
idx = tf.random.shuffle(idx)
x_train, y_train = tf.gather(data, idx[:train_num]), tf.gather(label, idx[:train_num])
x_test, y_test = tf.gather(data, idx[train_num:]) , tf.gather(label, idx[train_num:])


db_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
db_train = db_train.map(preprocess).shuffle(90000).batch(64)

db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
db_test = db_test.map(preprocess).shuffle(30000).batch(64)

sample = next(iter(db_train))


#cv+p+cv+p
model_layers = [
    # unit 1
    Conv1D(filters=8,kernel_size=4,kernel_initializer="he_uniform",strides=1,padding='valid',
                   use_bias=False,kernel_regularizer=regularizers.l2(0.0001)),
    MaxPool1D(pool_size=2),
    Conv1D(filters=16,kernel_size=6,kernel_initializer="he_uniform",strides=1,padding='valid',
                   use_bias=False,kernel_regularizer=regularizers.l2(0.0001)),
    MaxPool1D(pool_size=2),

    Flatten(),

    # fc*3
    layers.Dense(256, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(4, activation='softmax')

]


def main():
    model = Sequential(model_layers)

    model.build(input_shape=(None, 251, 1))
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['categorical_accuracy']
                  )
    model.fit(db_train, epochs=3, validation_data=db_test, validation_freq=2)
    model.evaluate(db_test)
    
    
    model.save('./model.h5')
    print('saved total model.')
    
    del model
    






if __name__ == '__main__':
    main()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 248, 8)            32        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 124, 8)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 119, 16)           768       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 59, 16)            0         
_________________________________________________________________
flatten (Flatten)            (None, 944)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               241920    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8

In [4]:
print('loaded model from file.')
model = tf.keras.models.load_model('./model.h5', compile=False)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['categorical_accuracy']
                      )

model.evaluate(db_test)

loaded model from file.
317/317 [==============================] - 2s 2ms/step - loss: 0.0504 - categorical_accuracy: 0.9877


[0.0504428930580616, 0.9876731634140015]